In [1]:
!pip install psx-data-reader
import psx 
import numpy as np
import pandas as pd
import keras
import scipy as sp
import sklearn
from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from datetime import date 
from dateutil.relativedelta import relativedelta


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class TrainModel():
  def __init__(self, ticker):
    self.ticker = ticker
    self.sc=None

  def get_train_data(self):
    now = date.today()
    five_year_earlier = (now + relativedelta(years=-5))
    end_date = (now + relativedelta(months=-2))
    data = psx.stocks(self.ticker, start=five_year_earlier, end=date.today())
    
    return data
  
  def get_test_data(self):
    now = date.today()
    two_months_earlier = (now + relativedelta(months=-2))
    data = psx.stocks(self.ticker, start=two_months_earlier, end=date.today())
    return data

  def atr_col(self,df):
    high_low = df['High'] - df['Low']
    high_prev_close = np.abs(df['High'] - df['Close'].shift())
    low_prev_close = np.abs(df['Low'] - df['Close'].shift())
    atr_df = pd.concat([high_low,high_prev_close,low_prev_close], axis=1)
    true_range = np.max(atr_df, axis=1)
    atr = true_range.rolling(14).mean()
    atr_df = atr.to_frame(name='ATR')
    ndf = pd.concat([df,atr_df],axis=1)
    ndf = ndf.dropna()
    return ndf

  def train(self):
    regressor = self.model()
    X_train, y_train, X_test, y_test = self.data_preprocess()
    regressor.fit(x = X_train, y = y_train, batch_size = 32, epochs = 100)
    regressor.save(f'/content/drive/MyDrive/Sproj/model/{self.ticker}.h5')

  def model(self):
    regressor = keras.Sequential()
    regressor.add(keras.layers.LSTM(units = 100, return_sequences = True, input_shape = (15, 6)))
    regressor.add(keras.layers.Dropout(rate = 0.2))
    regressor.add(keras.layers.LSTM(units = 100, return_sequences = True))
    regressor.add(keras.layers.Dropout(rate = 0.2))
    regressor.add(keras.layers.LSTM(units = 100, return_sequences = True))
    regressor.add(keras.layers.Dropout(rate = 0.2))
    regressor.add(keras.layers.LSTM(units = 100, return_sequences = True))
    regressor.add(keras.layers.Dropout(rate = 0.2))
    regressor.add(keras.layers.LSTM(units = 100, return_sequences = True))
    regressor.add(keras.layers.Dropout(rate = 0.2))
    regressor.add(keras.layers.LSTM(units = 100, return_sequences = False))
    regressor.add(keras.layers.Dropout(rate = 0.2))
    regressor.add(keras.layers.Dense(units = 6))
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return regressor

  def predict(self,model):
    pred_model = keras.models.load_model(model)
    sc = MinMaxScaler(feature_range = (0, 1))
    df = self.get_test_data()
    ndf = self.atr_col(df)
    pred_X = ndf.iloc[-15:].values
    pred_scaled = sc.fit_transform(pred_X)
    pred_scaled=pred_scaled.reshape(1,15,6)
    predicted_stock_price = pred_model.predict(pred_scaled)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    Open=predicted_stock_price[:,0][0]
    High=predicted_stock_price[:,1][0]
    Low=predicted_stock_price[:,2][0]
    Close=predicted_stock_price[:,3][0]
    ATR = predicted_stock_price[:,5][0]
    return Open,High,Low,Close,ATR
  
  def data_preprocess(self):
    df = self.get_train_data()
    ndf = self.atr_col(df)
    all_data_values = ndf.values
    train_X = all_data_values[:int(all_data_values.shape[0]*0.85), :]
    test_X = all_data_values[int(all_data_values.shape[0]*0.85):, :]
    print(train_X.shape , test_X.shape)
    self.sc = MinMaxScaler(feature_range = (0, 1))
    training_scaled = self.sc.fit_transform(train_X)
    testing_scaled = self.sc.fit_transform(test_X)
    X_train = []
    y_train = []
    for i in range(15, len(training_scaled)-7):
        X_train.append(training_scaled[i-15: i, :])
        y_train.append(training_scaled[i+7, :])
    X_train, y_train = np.array(X_train).astype(np.float64), np.array(y_train).astype(np.float64)
    print(X_train.shape , y_train.shape)
    X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],6)).astype(np.float64)
    print(train_X.shape,test_X.shape)
    temp = np.vstack((train_X , test_X))
    inputs = temp[len(temp)-len(test_X)- 7: ]
    inputs = inputs.reshape(-1, 6)
    inputs.shape
    inputs = self.sc.transform(inputs)
    X_test = []
    y_test = []
    for i in range(15, len(inputs)-7): 
        X_test.append(inputs[i-15: i, :])
        y_test.append(i+7)
        
    X_test = np.array(X_test).astype(np.float64)
    X_test = X_test.reshape((X_test.shape[0],  X_test.shape[1], 6))
    return X_train, y_train, X_test, y_test



In [13]:
test_model = TrainModel("HBL")
test_model.train()

(1042, 6) (185, 6)
(1020, 15, 6) (1020, 6)
(1042, 6) (185, 6)
Epoch 1/100


32/32 [==============================] - 11s 17ms/step - loss: 0.0232
Epoch 2/100
32/32 [==============================] - 1s 24ms/step - loss: 0.0119
Epoch 3/100
32/32 [==============================] - 1s 23ms/step - loss: 0.0101
Epoch 4/100
32/32 [==============================] - 1s 29ms/step - loss: 0.0095
Epoch 5/100
32/32 [==============================] - 1s 22ms/step - loss: 0.0099
Epoch 6/100
32/32 [==============================] - 1s 24ms/step - loss: 0.0097
Epoch 7/100
32/32 [==============================] - 1s 24ms/step - loss: 0.0099
Epoch 8/100
32/32 [==============================] - 1s 23ms/step - loss: 0.0091
Epoch 9/100
32/32 [==============================] - 1s 24ms/step - loss: 0.0089
Epoch 10/100
32/32 [==============================] - 1s 22ms/step - loss: 0.0087
Epoch 11/100
32/32 [==============================] - 1s 26ms/step - loss: 0.0086
Epoch 12/100
32/32 [==============================] - 1s 27ms/step - loss: 0.0082
Epoch 13/100
32/32 [================

In [14]:
Open,High,Low,Close,ATR = test_model.predict('/content/drive/MyDrive/Sproj/model/HBL.h5')

1/1 [==============================] - 3s 3s/step


In [15]:
print(Open,High,Low,Close,ATR)

76.256325 77.92351 75.19955 76.10345 2.9675472
